In [ ]:
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline

from quantopian.pipeline.filters import Q1500US, Q500US
from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.classifiers.morningstar import Sector, SuperSector


import pandas as pd
import numpy as np 

class ValueDayBeforeYesterday(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, values):
        out[:] = values[0]
        
class ValueDayBeforeYesterday_log(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, values):
        out[:] = np.log(values[0])

class MAValueChange(CustomFactor):
    def compute(self, today, assets, out, values):
        mean = pd.DataFrame(values).pct_change().mean()#.apply(np.log)
        out[:] = mean.values
        
    
        

def make_pipeline():
    pipe = Pipeline()
    
    base_universe = QTradableStocksUS()
    day_before_yesterday_close = ValueDayBeforeYesterday(inputs = [USEquityPricing.close])
    day_before_yesterday_volume = ValueDayBeforeYesterday(inputs = [USEquityPricing.volume])
    day_before_yesterday_volume_log = ValueDayBeforeYesterday_log(inputs = [USEquityPricing.volume])
    
    
    ma3_value_change = MAValueChange(inputs = [USEquityPricing.close], window_length=3)
    
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(98, 100)
    sector = Sector()
    
    pipe.add(day_before_yesterday_close, 'close_2days_before')
    pipe.add(day_before_yesterday_volume, 'volume_2days_before')
    pipe.add(USEquityPricing.close.latest, 'close_prev_day')
    pipe.add(ma3_value_change, 'ma3_value_change')
    pipe.add(sector, 'sector')
    pipe.add(np.log(day_before_yesterday_volume * day_before_yesterday_close), 'volume_2days_turnover_log')
    
    pipe.set_screen(high_dollar_volume & base_universe & (ma3_value_change < 0))
    return pipe

pipeline_results = run_pipeline(make_pipeline(), start_date='2017-1-1', end_date='2017-12-12')

In [ ]:
def fetch_todays_min_data(pipeline_results):
    dates = pipeline_results.index.get_level_values(0).unique()
    d = dict()
    for date in dates:
        tickers = pipeline_results.loc[date].index 
        d[date] = get_pricing(tickers, start_date=date, end_date=date, frequency='minute')
    return d

daily_min_data =  fetch_todays_min_data(pipeline_results)

In [ ]:
daily_min_data[daily_min_data.keys()[0]]


In [ ]:

def merge_all(pipeline_results,daily_min_data):
    dates = pipeline_results.index.get_level_values(0).unique()
    d = dict()
    for date in dates:
        d[date]  = merged(pipeline_results.loc[date],
                         daily_min_data[date])
    return d 
    
def merged(df_pipeline_results, pan_min_data):
    df_open_data = pan_min_data["open_price"]
    df_price = pan_min_data["price"]
    s_at_open = df_open_data.iloc[0].rename("09:30")
    s_at_entry = df_open_data.iloc[1].rename("09:31")
    s_at_1010 = df_price.iloc[39].rename("10:10")
    
    s_at_0940 = df_price.iloc[9].rename("09:40")
    s_at_0950 = df_price.iloc[19].rename("09:50")
    s_at_1000 = df_price.iloc[29].rename("10:00")
    s_at_1020 = df_price.iloc[49].rename("10:20")
    s_at_1030 = df_price.iloc[59].rename("10:30")
    
    
    df = pd.concat([df_pipeline_results, s_at_open, s_at_entry,
                    s_at_0940,s_at_0950,s_at_1000,
                    s_at_1010, s_at_1020,s_at_1030], axis=1)
    return df 

d = merge_all(pipeline_results,daily_min_data)

In [ ]:
d[daily_min_data.keys()[0]]

In [ ]:
def calc_gap(df_merged):
    df_merged["gap_from_day_before_yesterday"] = np.log(df_merged["09:30"] / df_merged["close_2days_before"])
    df_merged["gap_from_yesterday"] = np.log(df_merged["09:30"] / df_merged["close_prev_day"])
    df_merged["change_0930_1010"] = np.log(df_merged["10:10"] / df_merged["09:31"])
    df_merged["change_0930_1030"] = np.log(df_merged["10:30"] / df_merged["09:31"])
    
    df_merged["change_0930_0940"] = np.log(df_merged["09:40"] / df_merged["09:31"])
    df_merged["change_0930_0950"] = np.log(df_merged["09:50"] / df_merged["09:31"])
    df_merged["change_0930_1000"] = np.log(df_merged["10:00"] / df_merged["09:31"])
    df_merged["change_0930_1020"] = np.log(df_merged["10:20"] / df_merged["09:31"])
    df_merged["change_0930_1030"] = np.log(df_merged["10:30"] / df_merged["09:31"])
    
    
for date in d.keys():
    calc_gap(d[date])

In [ ]:
df = pd.concat(d)


In [ ]:
df.columns

In [ ]:
df[(df["gap_from_day_before_yesterday"] > 0.05)
  & (df['ma3_value_change'] > 0)].plot.scatter(x = 'gap_from_day_before_yesterday', y ='change_0930_1020', c = "ma3_value_change")

In [ ]:
df[(df["gap_from_day_before_yesterday"] > 0.05)][[u'change_0930_0940','change_0930_0950', 'change_0930_1000',
    'change_0930_1010', u'change_0930_1020', 'change_0930_1030']].describe().loc["mean"].plot(kind='bar')

In [ ]:
df[(df["gap_from_day_before_yesterday"] > 0.05) & (df['ma3_value_change'] < 0)].plot(
    kind = 'scatter', x = 'gap_from_day_before_yesterday', y ='change_0930_1010', c = 'sector')

In [ ]:
## Quantopian Research 

from quantopian.pipeline.data.builtin import USEquityPricing  
import pandas as pd  
from quantopian.pipeline import CustomFactor, Pipeline  
from quantopian.research import run_pipeline  
from quantopian.pipeline.filters import StaticSids

# SPYで検証
security = symbols(8554)  

def make_pipeline():
    # USEquityPricing の終値と始値の最新を取得してPipelineに流す
    latest_close = USEquityPricing.close.latest  
    latest_open = USEquityPricing.open.latest  
    return Pipeline(  
        columns = { 'Pipeline_close':latest_close,
                  'Pipeline_open':latest_open, },  
        screen = StaticSids([security])
    )

result = run_pipeline(make_pipeline(), '2017-11-30', '2017-12-13')
# multiindex を single index にする。
result = result.reset_index().set_index('level_0')  

# get_pricing を使って同期間のデータを取得
df=get_pricing(security, start_date='2017-11-30', end_date='2017-12-13',
            symbol_reference_date=None, frequency='daily', handle_missing='raise')
df = df.rename(columns = {"open_price": "get_pricing_open", "close_price": "get_pricing_close", })

# concatして比べる
pd.concat([result["Pipeline_open"], df["get_pricing_open"], result["Pipeline_close"], df["get_pricing_close"], ], axis=1)

